GRAPH ATTENTION NETWORKS

The following is the reimplementation and reproduction of results of the paper Graph Attention Networks by Petar Velickovic,Guillem Cucurull,Arantxa Casanova,Adriana Romero,Pietro Liò,Yoshua Bengio.

What is a graph attention network?
This neural network architecture operates on graph data and leveraging on graph convolutional networks and self attention layers allows to improve the accuracy of previous techniques on common graph datasets including the cora dateset,citeseet,pubmed and ppi.

Graph convolutional layer:

In the case of graph data the input is a graph with nodes that have a n dimensional vector representing its features and an adiacency matrix representing the connections between the different nodes of the graph.
In the case of graph convolutional layer to produce the hidden representation of a node x we would apply a sort of message passing ,where there are parameters to optimize ,between the nodes x and its neighbors, and this operation is done in parallel in all the nodes. 
The nodes features are all stacked in a matrix and this matrix is used to derive the hidden features for all the nodes using a W matrix of learnable weights and the adiacency matrix.
The weight matrix is where the learning takes place and could be descibed as a matrix or seen as a fully connected layer on the node embeddings.
The nodes features are obtained from the summation over all the neighbors features that are derived from the matrix multiplication with the weihgt matrix
The general formula to obtain the hidden features at layer i is:
\begin{equation}
h_{i}^{\prime}=\sigma\left(\sum_{j \in N(i)} W * h_{j}\right)
\end{equation}

Where W is the weight matrix,hj are the node embeddings from the last layer,N is the index of the neighbors of the node i and sigma is an activation function.
The aggregation function at the end can be a simple sum or for example an average or a different type of formula even one with learnable parameters.
In practise all of those operations are done in parallel using matrix multiplication and gpu for example in the pytorch geometric package and can be summarized as multiplying the features nodes with the weight matrix and then with the adiacency matrix and at the final step apply an activation function.

With convolutional layers we obtain a sort of message passing with learning parameters and the messages comes from the neighbors, thats why the number of convolutional layers is important because applying n convolutional layers allows to connect nodes n edges apart.

Self attention:
The convolutional layer can be estended using self attention.
One of the most important features of the graph attention network is the use of self attention, a mechanism commonly used in state of the art models in natural language processing , capable of learning how to give attention to different parts of the inputs to give the best possible results. 
The node i in this case not only has to work with its neighbors as in the graph convolutional layers but has also to learn attention coefficients to give to the neighboors of itself, to weight the features coming from them.

The attention coefficent that nodes i gives to node j can be seen as 
\begin{equation}
e_{i j}=a\left(W h_{i}, W h_{j}\right)
\end{equation}

where W is the parameters matrix and hi and hj are the nodes embeddings.

Before describing how the attention function a is implemented lets describe the normalization applied to the coefficients of node i neighboors:
this is the formula used to normaliza that is a simple softmax in order to normalize the coefficients of the neighbors to have a value between 0 and 1 and weightd by their importance :

\begin{equation}
\alpha_{i j}=\operatorname{softmax}_{j}\left(e_{i j}\right)=\frac{\exp \left(e_{i j}\right)}{\sum_{k e N(i)} \exp \left(e_{i k}\right)}=\frac{\exp \left(a\left(W h_{i}, W h_{j}\right)\right)}{\sum_{k e N(i)} \exp \left(a\left(W h_{i}, W h_{j}\right)\right)}
\end{equation}

the softmax function firstly exponentiates all the eij attention coefficients to obtain values between 0 and 1 and then returns the percentage of its attention with respect to the attention given to all the neighboors togheter.

Now let's see how the attention coefficients aij are calculated.
There are different ways to get the attention coefficients, for example in the original paper attention is all you neeed its calculated by using the dot product of the features learned applying the a linear trasformation to the embeddings.

However in this paper they opted for a different type of attention mechanism:

they firstly stack the features obtained by multiplying the embeddings with W and then they apply a fully connected layer to get a value that is then passed by a leaky relu activation function before the softmax normalization.
The formula applied at the end to obtain the coefficients normalized is the follwoing:

\begin{equation}
\alpha_{i j}=\frac{\exp \left(\text { LeakyReLU }\left(\overrightarrow{w_{a}^{T}}\left[W h_{i} \| W h_{j}\right]\right)\right)}{\sum_{k \in N(i)} \exp \left(\text { LeakyReLU }\left(\overrightarrow{w_{a}^{T}}\left[W h_{i} \| W h_{j}\right]\right)\right)}
\end{equation}

the LeakyReLu is used to cut off all the negatives values by a large amount but the entire one and is helpful to emphatize positive relationships found by the model.

graph attention network:

merging the things seen before allow to implement the graph attention layer where there is the use of the graph convolutional layer and the attention coefficients to weight the neighbors messages like a linear combination.
Here is the formula that describes how to obtain the node features:

\begin{equation}
h_{i}^{\prime}=\sigma\left(\sum_{j \in N(i)} \alpha_{i j} W * h_{j}\right)
\end{equation}

the aij are values between 0 and 1 that sums to one and are applied in the summation before the activation function

To summarize the learnable weights are the ones of the matrix W used to transform the embeddings and the ones of the attention mechanism (fully connected layer).
In practise we use multihead attention ,so we learn multiple attention coefficients between i and j and we use those to weight the message passing of the neightboors and obtain the features of the output.
 

In [ ]:
import torch
from torch_geometric.nn import GCNConv, GATv2Conv
import torch.nn.functional as F

#this is the graph attention network module
class GraphAttentionNetwork(torch.nn.Module):
  """Graph Attention Network"""
  def __init__(self, dim_in, dim_h, dim_out, heads=8):
    super().__init__()
    #we have 2 graph attention layers specifying dimension of inputs,outputs and heads
    self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
    self.gat2 = GATv2Conv(dim_h*heads, dim_out, heads=1)
  def forward(self, data):
    #i get the input features and the graph connections as input x
    x, edge_index=data.x,data.edge_index
    #i filter the input features with a dropout layer avoid overfitting and improve generalization
    h = F.dropout(x, p=0.6, training=self.training)
    #i apply the first graph attention layer
    h = self.gat1(x, edge_index)
    #i apply a elu activation function as in the paper
    h = F.elu(h)
    #i filter the features h with a dropout layer avoid overfitting and improve generalization
    h = F.dropout(h, p=0.6, training=self.training)
    #i apply the second graph attention layer
    h = self.gat2(h, edge_index)
    #i return the logsoftmax of the output to normalize the output
    return h

#this is the graph convolutional network module
class GraphConvolutionalNetwork(torch.nn.Module):
    def __init__(self, in_size, h_size,out_size):
        super().__init__()
        #we have 2 graph convolutional layers specifying dimension of inputs,outputs and hiddend features
        self.conv1 = GCNConv(in_size, h_size)
        self.conv2 = GCNConv(h_size, out_size)

    def forward(self, data):
        #i get the input features and the graph connections as input x
        x, edge_index = data.x, data.edge_index
        #dropout layer to avoid overfitting and improve generalization
        x = F.dropout(x, p=0.6, training=self.training)
        #i apply the first graph convolutional layer
        x = self.conv1(x, edge_index)
        #i apply a relu activation function 
        x = F.relu(x)
         #i filter the features x with a dropout layer avoid overfitting and improve generalization
        x = F.dropout(x, p=0.6, training=self.training)
        #i apply the second graph convolutional layer
        x = self.conv2(x, edge_index)
        #i return the logsoftmax of the output to normalize the output
        return x


class Mlp(torch.nn.Module):
    def __init__(self, in_size, h_size,out_size):
        super().__init__()
        #we have 2 graph convolutional layers specifying dimension of inputs,outputs and hiddend features
        self.fc1 = torch.nn.Linear(in_size, h_size)
        self.fc2 = torch.nn.Linear(h_size, out_size)

    def forward(self, data):
        #i get the input features and the graph connections as input x
        x, edge_index = data.x, data.edge_index
        #dropout layer to avoid overfitting and improve generalization
        x = F.dropout(x, p=0.6, training=self.training)
        #i apply the first graph convolutional layer
        x = self.fc1(x)
        #i apply a relu activation function 
        x = F.relu(x)
         #i filter the features x with a dropout layer avoid overfitting and improve generalization
        x = F.dropout(x, p=0.6, training=self.training)
        #i apply the second graph convolutional layer
        x = self.fc2(x)
        #i return the logsoftmax of the output to normalize the output
        return x

In [ ]:
from torch_geometric.datasets import PPI
import torch
from torch_geometric.loader import DataLoader
from nn import GraphAttentionNetwork, GraphConvolutionalNetwork, Mlp
import wandb
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F


class Trainer:
    def __init__(self,model,optimizer,epochs,datasetName):
        self.model=model
        self.optimizer=optimizer
        self.epochs=epochs
        self.loadDataset(datasetName)

    #training loop
    def train(self):
        it=0
        self.model.train()
        for epoch in range(self.epochs):
            it+=1    
            self.optimizer.zero_grad()    
            # if the database is ppi then the task is the multilabeling classification 
            #otherwise is node classification
            if self.datasetName=="ppi":
                out = model(self.dataset[0])
                #the loss in this case is the binary cross entropy with logits loss
                #the logits are numbers between -inf and inf not yet normalized for every class
                #in this case i have multiple graphs so i just pass the first graph to train on
                #i could improve it to train on all the others like in a kfold cross validation
                loss=torch.nn.BCEWithLogitsLoss(reduction='mean')(out, self.dataset[0].y)
            else:
                out = model(self.dataset)
                #the loss here is the negative log likelihood 
                #i mask the graph since i have just one graph to train and then i evaluate with a different mask
                #i softmax the output to apply the nll loss
                out=F.log_softmax(out, dim=1)
                loss = F.nll_loss(out[self.dataset.train_mask], self.dataset.y[self.dataset.train_mask])
            #backpropagation
            loss.backward()
            self.optimizer.step()
            print("mean_loss_train",loss.data.mean())
            wandb.log({"mean_loss_train":loss.data.mean()})
            #i evaluate the model on the test part of the graph
            self.test()

    def test(self):
        print("testing phase")
        model.eval()
        #if the dataset is ppi then the task is the multilabeling classification
        #so i get the output and then i apply the sigmoid to get the probability of each class
        #i threshold the probability to get the class for each label
        #then i compare the output with the ground truth
        with torch.no_grad():
            if self.datasetName=="ppi":
                #i evaluate the model on the second graph of the dataset not the first of the trainign
                pred = model(self.dataset[1])
                y=self.dataset[1].y
                loss=torch.nn.BCEWithLogitsLoss(reduction='mean')(pred, y)
                pred=F.sigmoid(pred)
                pred=torch.where(pred>0.5,1,0)
                correct=(pred==y).sum()
                acc=correct/len(y)
                # correct = (pred == y).sum()
                # # acc=F1Score(121,0)(pred,y)
                # acc = int(correct) / int(self.dataset[1].y.shape[0])
            else:
                #if the dataset is cora, citeseer or pubmed then the task is the node classification
                #so i have the nll loss at the end on the log softmax output
                pred = model(self.dataset)
                #i softmax the prdictions and i apply the negative log likelihood to the test graph (masked)and the argmax to get the class
                pred=F.log_softmax(pred, dim=1)
                loss = F.nll_loss(pred[self.dataset.test_mask], self.dataset.y[self.dataset.test_mask])
                #calculate accuracy
                pred=pred.argmax(dim=1)
                correct = (pred[self.dataset.test_mask] == self.dataset.y[self.dataset.test_mask]).sum()
                acc = int(correct) / int(self.dataset.test_mask.sum())

        #log the results
        print("mean_loss_test",loss.data.mean(),"mean_accuracy_test:",acc)
        wandb.log({"mean_loss_test":loss.data.mean(),"mean_accuracy_test":acc})


    #this function loads the dataset based on the name 
    def loadDataset(self,datasetName="cora"):
        self.datasetName = datasetName
        #should be cora, citeseer and pubmed or ppi
        if datasetName=="ppi":
            self.dataset=PPI(root="./data/ppi")
        else:
            self.dataset=Planetoid(name=datasetName,root="./data/"+datasetName)
        #print the dataset generalities
        print("name of dataset: {},number of classes: {},number of nodes features: {},number of graphs: {}".format(self.dataset,self.dataset.num_classes,self.dataset.num_node_features,len(self.dataset)))
        if datasetName!="ppi":
            self.dataset=self.dataset[0]


if __name__ == "__main__":
    # this is the number of features of the input and the output dimension that the model will have based on the dataset
    inOut={
        "cora":(1433,7),
        "citeseer":(3703,6),
        "pubmed":(500,3),
        "ppi":(50,121)
    }
    # i set the name of the dataset here and the type of model
    datasetName="pubmed"
    type="gcn"
    device = torch.device('cpu')
    #i create the model based on the settings of dataset name and type
    if type=="gcn":
        model=GraphConvolutionalNetwork(inOut[datasetName][0],13,inOut[datasetName][1])
    elif type=="gat":
        model=GraphAttentionNetwork(inOut[datasetName][0],13,inOut[datasetName][1])
    elif type=="mlp":
        model=Mlp(inOut[datasetName][0],50,inOut[datasetName][1])
    # i create the the wandb project and the run    
    name=type+" "+datasetName
    run=wandb.init(project='graphAttentionNetwork', entity='bbooss97',name=name)
    model.to(device)
    run.watch(model)
    #define the settings for the training
    epochs=200
    optmimizer=torch.optim.Adam(model.parameters())
    #create the trainer and start the training process
    trainer=Trainer(model,optmimizer,epochs,datasetName=datasetName)
    trainer.train()
